# Imports

In [1]:
%reload_ext nb_black
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly
import seaborn as sns

%matplotlib inline
from scipy import stats

plt.style.use(["dark_background"])
from matplotlib.patches import Ellipse
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn import datasets, metrics

<IPython.core.display.Javascript object>

In [2]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "heartdisease"
conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"
query = """
SELECT *
FROM heartdisease
"""
heartdisease_df = pd.read_sql_query(query, con=conn_str)

# df formatting
# Define the features and the outcome
X = heartdisease_df.iloc[:, :13]
y = heartdisease_df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace="?", value=0)

# Binarize y so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

<IPython.core.display.Javascript object>

In [5]:
# scale the data
scaled_X = StandardScaler().fit_transform(X)

<IPython.core.display.Javascript object>

> 1. Apply GMM to the heart disease dataset by setting n_components=2. Get ARI and silhouette scores for your solution and compare it with those of the k-means and hierarchical clustering solutions that you implemented in the previous checkpoint assignments. Which algorithm performs best?

In [6]:
# GMM model object
gmm_cluster = GaussianMixture(n_components=2)

# fit the model to tha data
clusters = gmm_cluster.fit_predict(scaled_X)

<IPython.core.display.Javascript object>

In [8]:
print(
    "Adjusted Rand Index of the GMM solution: {}".format(
        metrics.adjusted_rand_score(y, clusters)
    )
)
print(
    "The silhoutte score of the GMM solution: {}".format(
        metrics.silhouette_score(scaled_X, clusters, metric="euclidean")
    )
)

Adjusted Rand Index of the GMM solution: 0.18389186035089963
The silhoutte score of the GMM solution: 0.13628813153331445


<IPython.core.display.Javascript object>

> The 2 cluster KMeans ARI was higher but the silhouette score was slightly lower than the GMM.
> The hierarchical ARI and silhouette score were higher than the GMM

> 2. GMM implementation of scikit-learn has a parameter called covariance_type. This parameter determines the type of covariance parameters to use. There are four types that you can specify:

* full: This is the default. Each component has its own general covariance matrix.
* tied: All components share the same general covariance matrix.
* diag: Each component has its own diagonal covariance matrix.
* spherical: Each component has its own single variance.

In [13]:
cov = ["full", "tied", "diag", "spherical"]

for param in cov:
    # GMM model object
    gmm_cluster = GaussianMixture(n_components=2, covariance_type=param)

    # fit the model to tha data
    clusters = gmm_cluster.fit_predict(scaled_X)

    print(
        f"Adjusted Rand Index of the {param} GMM solution: {metrics.adjusted_rand_score(y, clusters)}"
    )

    print(
        f"The silhoutte score of the {param} GMM solution: {metrics.silhouette_score(scaled_X, clusters, metric='euclidean')}"
    )

Adjusted Rand Index of the full GMM solution: 0.4207322145049338
The silhoutte score of the full GMM solution: 0.16118591340148433
Adjusted Rand Index of the tied GMM solution: 0.18389186035089963
The silhoutte score of the tied GMM solution: 0.13628813153331445
Adjusted Rand Index of the diag GMM solution: 0.37878842718089933
The silhoutte score of the diag GMM solution: 0.15836933745078682
Adjusted Rand Index of the spherical GMM solution: 0.20765243525722465
The silhoutte score of the spherical GMM solution: 0.12468753110276873


<IPython.core.display.Javascript object>

> The full covariance type is the best